# Using lists to implement Stack

## ``lists`` are everywhere
- The first aggregate structure used
    - ``append, insert``
    - ``index``
    - ``remove, del``
- Brackets [...]
- Underlying array implementation

### Example: List of Fibonacci numbers

In [2]:
def fibonacci(n):
    """Return n>=2 Fibonacci numbers."""
    a = b = 1
    result = [a, b]
    while n > 2:
        n = n - 1
        a, b = b, a + b
        result.append(b)
    return result

In [4]:
fibonacci(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [5]:
fibonacci(100)

[1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181,
 6765,
 10946,
 17711,
 28657,
 46368,
 75025,
 121393,
 196418,
 317811,
 514229,
 832040,
 1346269,
 2178309,
 3524578,
 5702887,
 9227465,
 14930352,
 24157817,
 39088169,
 63245986,
 102334155,
 165580141,
 267914296,
 433494437,
 701408733,
 1134903170,
 1836311903,
 2971215073L,
 4807526976L,
 7778742049L,
 12586269025L,
 20365011074L,
 32951280099L,
 53316291173L,
 86267571272L,
 139583862445L,
 225851433717L,
 365435296162L,
 591286729879L,
 956722026041L,
 1548008755920L,
 2504730781961L,
 4052739537881L,
 6557470319842L,
 10610209857723L,
 17167680177565L,
 27777890035288L,
 44945570212853L,
 72723460248141L,
 117669030460994L,
 190392490709135L,
 308061521170129L,
 498454011879264L,
 806515533049393L,
 1304969544928657L,
 2111485077978050L,
 3416454622906707L,
 5527939700884757L,
 8944394323791464L,
 14472334024676221L,
 23416728348467685L,
 37889062373143906L,
 61305790721611

## Underlying Array Implementation
- Potential to waste resources as size grows
    - every element of **list** is a reference
- Operations can become quite expensive
    - ``del myList[0]`` is an O(n) operation!
        - Must move remaining ``n - 1`` elements
- Use wisely to ensure efficient performance

## Use List as a Stack
- Stack is abstract data type
    - ``push(v)`` to add value to top of stack
    - ``pop()`` removes topmost value and returns it
    - ``isEmpty()`` determines if stack is empty
- Protects against inadvertent updates
- Performance
    - ``isEmpty`` O(1)
    - ``push``    O(1)
    - ``pop``     O(1)
- Amortized cost

In [7]:
class Stack:
    def __init__(self):
        """Demonstrate using list as storage for a Stack."""
        self.stack = []
        
    def isEmpty(self):
        """Determines wheter stack is empty."""
        return len(self.stack) == 0
    
    def push(self, v):
        """Push v onto the stac. O(1) performance."""
        self.stack.append(v)
        
    def pop(self):
        """Remove topmost element and return it. O(1) performance."""
        if self.isEmpty():
            raise Exception('Stack is empty')
        return self.stack.pop()
    def __repr__(self):
        """Show representation"""
        return "stack:" + str(self.stack)

In [8]:
s = Stack()

In [16]:
s.push(1)

In [17]:
s.push(2)

In [18]:
s

stack:[1, 2]

In [19]:
s.pop()

2

In [20]:
s.pop()

1

In [21]:
s.isEmpty()

True

# Principle: Separate Structure from Function

## Problem: Process infinite data series
- Given limitless series of data events
    - **Volume**: Much too large to store in memory
    - **Time**: Highe-speed arrival of data
- Examples
    - Stock or currency prices
    - Sensor data

## Example: Compute Moving Average Efficiently in a Window
- Given series of data events over time
    - Process fixed number of values in **sliding window**
    - Moving average defined over values in window  

### Naive Python solution
- Use ``window`` **list** to store values in window
    - Append new ``value`` to end of list
    - Remove 0th element if length exceeds window ``size``

In [22]:
def add(window, value, size):
    if len(window) == size:
        del window[0] # SIGNIFICANTLY INEFFICIENT OPERATION!!!
    window.append(value)

- Never forget that a Python **list** is an array
    - Remove 0th element must copy n - 1 elements
    - Thus ``del window[0]`` is an O(n) operation
    - Eliminate this cost with a **CircularBuffer**

### Circular Buffer Solution
- Use Python **list** as fixed-size buffer
    - Maintain illusion that it wraps around to form a (clockwise) circle
    - Queue behaviour
        - Add to the end
        - Remove from the front
        - These operations become O(1) always!
- Adding elements overwrites oldest element in buffer
    - Maintain (low, high) indices within buffer
    
#### Circular Buffer Data Type Definition
- Fixed-size storage
- Separate indices and count
    - When not empty, ``low`` is index of first element
    - When not empty, ``high`` is index of **next location** to use
- Read-only operations
    - ``isEmpty, isFull, __len__`` O(1)
- Modifying
    - ``add(v)`` O(1)
    - ``remove()`` O(1)
- Iterator used by **in** O(n)

In [50]:
class CircularBuffer:
    def __init__(self, size):
        """Construct fixed size buffer"""
        self.size = size
        self.buffer = [None]*size
        self.low = 0
        self.high = 0
        self.count = 0
        
    def isEmpty(self):
        return self.count == 0
    
    def isFull(self):
        return self.count == self.size
    
    def add(self, value):
        if self.isFull():
            self.low = (self.low + 1) % self.size
        else:
            self.count += 1
        self.buffer[self.high] = value
        self.high = (self.high + 1) % self.size
            
    def remove(self):
        if self.count == 0:
            raise Exception("Circular buffer is empty!")
        value = self.buffer[self.low]
        self.low = (self.low + 1) % self.size
        self.count -= 1
        return value
    
    def __iter__(self):
        idx = self.low
        num = self.count
        while num > 0:
            yield self.buffer[idx]
            idx = (idx + 1) % self.size
            num -= 1
    
    def __repr__(self):
        if self.isEmpty():
            return 'cb:[]'
        return 'cb:[' + ','.join(map(str,self)) + ']'

In [61]:
c = CircularBuffer(5)
c

cb:[]

In [62]:
c.add(5)
c

cb:[5]

In [63]:
c.add(7)
c.add(9)
c.add(11)
c.add(13)
c

cb:[5,7,9,11,13]

In [64]:
c.isFull()

True

In [65]:
c.add(15)
c

cb:[7,9,11,13,15]

In [66]:
c.buffer

[15, 7, 9, 11, 13]

In [67]:
c.low

1

In [68]:
c.high

1

In [69]:
c.remove()

7

In [75]:
c

cb:[11,13,15,11,13]

# Project: Moving Average
You can extend the circular buffer to efficiently compute moving average computations over large data set. With some changes, it is possible to extend this same concept to compute the standard deviation over a data series within a given window once the formula for standard deviation (i.e., stdev) is converted into the appropriate form.

- Construct efficient moving average computation
    - Create ``MovingAverage`` class **extending** ``CircularBuffer``
    - Ability to define new classes from existing ones

In [94]:
class MovingAverage(CircularBuffer):
    
    def __init__(self, size):
        """Store buffer in given storage."""
        CircularBuffer.__init__(self, size)
        self.total = 0
        
    def getAverage(self):
        """Returns moving average (zero if no elements)"""
        if self.count == 0:
            return 0
        return self.total/self.count
    
    def remove(self):
        """Removes oldest value from non-empty buffer."""
        removed = CircularBuffer.remove(self)
        self.total -= removed
        return removed
    
    def add(self, value):
        """Adds value to buffer, overwrite as needed."""
        if self.isFull():
            delta = -self.buffer[self.low]
        else:
            delta = 0
        delta += value
        self.total += delta
        CircularBuffer.add(self, value)
        
    def __repr__(self):
        """String representation of moving average."""
        if self.isEmpty():
            return 'ma:[]'
        return 'ma:[' + ','.join(map(str, self)) + ']: ' + str(1.0*self.total/self.count)

In [100]:
ma = MovingAverage(3)
ma

ma:[]

In [101]:
ma.add(4)
ma

ma:[4]: 4.0

In [102]:
ma.add(8)
ma

ma:[4,8]: 6.0

In [103]:
ma.add(16)
ma

ma:[4,8,16]: 9.33333333333

In [104]:
ma.total

28

In [105]:
ma.remove()

4

In [106]:
ma

ma:[8,16]: 12.0

# Python ``list`` Primary Weaknesses
- Avoid functions that return **list** objects whose values are simply iterated
    - Consider **generator**
        - Generator avoids constructing lists
        - When can compute the elements using computation

In [107]:
def fibonacciGen(n):
    a = b = 1
    yield a
    yield b
    while n > 2:
        n = n - 1
        a, b = b, a + b
        yield b

In [109]:
for _ in fibonacciGen(100):
    print(_)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578
5702887
9227465
14930352
24157817
39088169
63245986
102334155
165580141
267914296
433494437
701408733
1134903170
1836311903
2971215073
4807526976
7778742049
12586269025
20365011074
32951280099
53316291173
86267571272
139583862445
225851433717
365435296162
591286729879
956722026041
1548008755920
2504730781961
4052739537881
6557470319842
10610209857723
17167680177565
27777890035288
44945570212853
72723460248141
117669030460994
190392490709135
308061521170129
498454011879264
806515533049393
1304969544928657
2111485077978050
3416454622906707
5527939700884757
8944394323791464
14472334024676221
23416728348467685
37889062373143906
61305790721611591
99194853094755497
160500643816367088
259695496911122585
420196140727489673
679891637638612258
1100087778366101931
1779979416004714189
2880067194370816120
4660046610375530309
7540113804746346429
